# 🔬 Multi-Agent RAG Research Assistant

An interactive and modular Retrieval-Augmented Generation (RAG) pipeline that combines PDF/text ingestion, FAISS vector retrieval, and LLM-based summarization using Together.ai’s LLaMA-4. Built with LangGraph agents and deployed via Gradio for real-time, user-customizable research summarization.


## ⚙️ Tech Stack

- **LangGraph** – Multi-agent state machine orchestration
- **FAISS** – Fast Approximate Nearest Neighbor vector store
- **Hugging Face Sentence Transformers** – Text embeddings (MiniLM)
- **Together.ai** – Hosted LLaMA-4 summarization
- **SerpAPI** – Google Search snippets
- **Gradio** – User-friendly interface


In [7]:
# Basic NLP and retrieval tools
!pip install -q sentence-transformers langchain faiss-cpu serpapi

In [5]:
# LLaMA model loader (this is the slow one that may need C++ compilation)
!pip install -q llama-cpp-python


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.49
    Uninstalling langchain-core-0.3.49:
      Successfully uninstalled langchain-core-0.3.49
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [8]:
!pip install llama-cpp-python[server]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00


In [9]:
!pip install together


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 2.1 MB/s eta 0:00:00


In [10]:
!pip install langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 9.2 MB/s eta 0:00:00


In [11]:
!pip install grandalf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.7 MB/s eta 0:00:00


In [12]:
!pip install gradio
!python app.py


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 51.7 MB/s eta 0:00:00
python3: can't open file '/content/app.py': [Errno 2] No such file or directory


In [13]:
import faiss
import numpy as np
import requests
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.docstore.document import Document

## 🧪 Pipeline Flow

1. 🔍 **SearchAgent** – Uses SerpAPI to fetch top Google results
2. 📄 **PDFLoaderAgent** – Loads and splits uploaded PDFs into text
3. 🧠 **EmbedAgent** – Embeds all documents using MiniLM
4. 🔎 **RetrieveAgent** – Uses FAISS to find top-k relevant chunks
5. 📝 **SummarizeAgent** – Calls Together.ai LLaMA-4 with user prompt

The entire workflow is managed via LangGraph's stateful graph execution.


## 📥 How to Use

1. Enter your **research query** (e.g., "AutoML in 2024 (500-word summary)")
2. Optionally upload a **PDF file**
3. Enter your **SerpAPI** and **Together.ai API keys**
4. Click **Submit** and wait for the generated summary


In [14]:
def search_serpapi(query, api_key):
    params = {
        "engine": "google",
        "q": query,
        "api_key": api_key,
        "num": 20  # Ask for more results
    }
    response = requests.get("https://serpapi.com/search", params=params)
    data = response.json()
    results = []
    for item in data.get("organic_results", []):
        results.append(item.get("snippet", ""))
    return results

# -----------------------------
# 2. Query SerpAPI for NAS content
# -----------------------------
SERPAPI_KEY = "e63948d7aaba37cb53c7f84f9a8e8e2e2985470ad2bb00937fc2be10aabf13ad"
query = "NAS (Neural Architecture Search) Techniques in 2024"
snippets = search_serpapi(query, SERPAPI_KEY)

print("🔎 Retrieved snippets:")
for i, s in enumerate(snippets, 1):
    print(f"{i}. {s}")
print(f"\n✅ Total snippets retrieved: {len(snippets)}")


🔎 Retrieved snippets:
1. One famous implementation is Google's AutoML, where RL agents design neural networks for tasks like image classification.
2. This paper delves into the multifaceted aspects of NAS, elaborating on its recent advances, applications, tools, benchmarks and prospective research directions.
3. This work presents a novel approach to neural architecture search (NAS) that aims to increase carbon efficiency for the model design process ...
4. Neural architecture search (NAS) is an automated machine learning method that aims to find optimal model structures by searching the neural network architecture ...
5. Neural Architecture Search (NAS) is a technique in machine learning that automates the process of designing neural network architectures.
6. NAS with Reinforcement Learning. Reinforcement learning-based NAS methods frame the architecture search as a sequential decision-making process.
7. We propose a novel and effective method called Evolutionary Gradient-Based Neural

In [15]:
docs = [Document(page_content=s) for s in snippets if s.strip() != ""]

# Ensure we have enough to train IVF index (e.g., at least nlist)
if len(docs) < 5:
    raise ValueError("Not enough snippets to train IndexIVFFlat. Please retrieve more results.")


## 🔐 API Keys Required

- `SERPAPI_KEY` from [serpapi.com](https://serpapi.com/)
- `TOGETHER_API_KEY` from [platform.together.xyz](https://platform.together.xyz)

Keys can be provided in the Gradio UI or loaded securely in a `.env` file (if running locally).


In [16]:
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores.faiss import FAISS
from langchain.docstore.document import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
import faiss
import numpy as np

# 1. Documents & embeddings
docs = [Document(page_content=s) for s in snippets if s.strip() != ""]
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectors = embedding_model.embed_documents([doc.page_content for doc in docs])
vectors_np = np.array(vectors).astype("float32")

# 2. Build IVF index
d = vectors_np.shape[1]
nlist = 5
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)
index.train(vectors_np)
index.add(vectors_np)

# 3. LangChain expects this mapping:
index_to_docstore_id = {i: str(i) for i in range(len(docs))}
docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(docs)})

# 4. Now construct FAISS object manually
vectorstore = FAISS(
    embedding_function=embedding_model.embed_query,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# 5. Run similarity search
query_text = "recent advancements in neural architecture search"
results = vectorstore.similarity_search(query_text, k=3)

print("\n📄 Top relevant documents:")
for i, doc in enumerate(results, 1):
    print(f"\n--- Retrieved {i} ---\n{doc.page_content}")


<ipython-input-16-58ea2205578f>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warni

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


📄 Top relevant documents:

--- Retrieved 1 ---
We propose a novel and effective method called Evolutionary Gradient-Based Neural Architecture Search (EG-NAS).

--- Retrieved 2 ---
Neural Architecture Search (NAS) methods seek effective optimization toward performance metrics regarding model accuracy and generalization while facing ...

--- Retrieved 3 ---
Neural Architecture Search (NAS) aims to automate deep neural network design across various applications, while a good search space design is core to NAS ...


In [17]:
from together import Together

client = Together(api_key="f686a109d1f4267d0019648d3b74d2f014a5c8585ec58e9734e98c6b38f1f658")

retrieved_texts = [doc.page_content for doc in results]
prompt = "Summarize the following research snippets:\n\n" + "\n\n".join(retrieved_texts)

response = client.chat.completions.create(
    model="meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8",
    messages=[
        {"role": "system", "content": "You are a helpful research assistant."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=512,
    temperature=0.7,
)

print("\n✅ Summary:\n", response.choices[0].message.content)



✅ Summary:
 Here is a summary of the research snippets:

The snippets discuss Neural Architecture Search (NAS), a method that aims to automate the design of deep neural networks. The researchers propose a new NAS method called Evolutionary Gradient-Based Neural Architecture Search (EG-NAS), suggesting that existing NAS methods face challenges in optimizing model accuracy and generalization. A key aspect of NAS is designing a good search space, which is crucial for its effectiveness.


## 🔐 API Keys Required

- `SERPAPI_KEY` from [serpapi.com](https://serpapi.com/)
- `TOGETHER_API_KEY` from [platform.together.xyz](https://platform.together.xyz)

Keys can be provided in the Gradio UI or loaded securely in a `.env` file (if running locally).
